In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("facet-terms-for-082a_ClassificationDdc.csv")

In [8]:
df.describe(include='all')

,term,count
count,45487,45487.000000
unique,45487,NaN
top,2,NaN
freq,1,NaN
mean,NaN,9.490734
std,NaN,95.892351
min,NaN,1.000000
25%,NaN,1.000000
50%,NaN,1.000000
75%,NaN,2.000000


It is visible from the above stats that there are 45487 rows of data which are all unique.

In [9]:
df_simple_term_resolution = pd.read_csv("combined_output.csv")

In [10]:
non_null_count = df_simple_term_resolution['Returned List'].notnull().sum()
total_count = len(df)
percentage_non_null = (non_null_count / total_count) * 100

print(f"Percentage of terms with values in 'Returned List': {percentage_non_null:.2f}%")


Percentage of terms with values in 'Returned List': 100.00%


All the rows have returned values which means API returned some response for all the rows.

In [11]:
valid_responses = df_simple_term_resolution['Returned List'].apply(lambda x: x != '[]' and pd.notnull(x))

valid_count = valid_responses.sum()

empty_count = (~valid_responses).sum()

total_count = len(df)


print(f"Total rows: {total_count}")
print(f"Number of rows with valid responses: {valid_count}")
print(f"Number of rows with empty lists: {empty_count}")


percentage_valid = (valid_count / total_count) * 100
percentage_empty = (empty_count / total_count) * 100

print(f"Percentage of rows with valid responses: {percentage_valid:.2f}%")
print(f"Percentage of rows with empty lists: {percentage_empty:.2f}%")

Total rows: 45487
Number of rows with valid responses: 10725
Number of rows with empty lists: 40465
Percentage of rows with valid responses: 23.58%
Percentage of rows with empty lists: 88.96%


The above stats show us that 23.58% of terms were resolved using the first API while 76.42% of terms were not resolved.

Now lets load and analyze the data after finding the gap for valid terms after first API

In [16]:
df_with_gap= pd.read_csv("combined_output_with_gap.csv")
df_with_valid_reponse_from_API= df_simple_term_resolution[df_simple_term_resolution['Returned List'].apply(lambda x: len(str(x))>20)]

Merging these two dataframes to find out percentage of valid terms with gap.

In [21]:
df_with_valid_reponse_from_API['term'] = df_with_valid_reponse_from_API['term'].astype(str)
df_with_gap['term'] = df_with_gap['term'].astype(str)

/tmp/ipykernel_28621/1664961194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_valid_reponse_from_API['term'] = df_with_valid_reponse_from_API['term'].astype(str)


In [29]:
df_with_gap_unique = df_with_gap.drop_duplicates(subset='term')


In [30]:
df_merged_for_gap= pd.merge(df_with_valid_reponse_from_API, df_with_gap_unique[['term', 'HasGap']], on='term', how='left')

In [33]:
count_values = df_merged_for_gap['HasGap'].value_counts()
percentage_values = df_merged_for_gap['HasGap'].value_counts(normalize=True) * 100
results = pd.DataFrame({
    'Count': count_values,
    'Percentage': percentage_values
})

print(results)

        Count  Percentage
HasGap                   
False    6396   66.266059
True     3256   33.733941


There for 66.37% of valid terms after the first API resolution didn't have the gap while 33.73 has the gap.

Know lets move onto the second API that also resolves the complex terms from the auxilary tables.

Lets load the data after resolving the unresolved terms from the simple API using complex API.

In [34]:
df_complex_term_resolution= pd.read_csv("combined_output_with_new_API.csv")

In [35]:
df_complex_term_resolution

,Unnamed: 0,term,Response Code,Returned List,count,Returned from second API
0,0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0,[{'uri': 'http://dewey.info/class/000.00/e23/'...
1,1,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0,[{'uri': 'http://dewey.info/class/000.00/e23/'...
2,2,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0,[{'uri': 'http://dewey.info/class/000.00/e23/'...
3,3,0,200.0,[],12.0,[{'uri': 'http://dewey.info/class/000.00/e23/'...
4,4,00,NaN,NaN,1.0,[]\n
...,...,...,...,...,...,...
53399,53399,zot,200.0,[],1.0,NaN
53400,53400,zs95f,200.0,[],1.0,NaN
53401,53401,zur,200.0,[],1.0,NaN
53402,53402,zw315m,200.0,[],1.0,NaN
